In [1]:
import numpy as np 
import pandas as pd 

In [2]:
import os 
inputFolder = '../Research Work/Docs/' 
for root, directories, filenames in os.walk(inputFolder): 
    for filename in filenames: 
        print(os.path.join(root,filename))
        with open(os.path.join(root, filename)) as f:
            line_count = 0
            id_set = set()
            for l in f.readlines():
                line_count += 1
                if filename == "CISI.REL":
                    id_set.add(l.lstrip(" ").split(" ")[0])
                elif l.startswith(".I "):
                    id_set.add(l.split(" ")[1].strip())
            print(f"{filename} : {len(id_set)} items, over {line_count} lines.")

../Research Work/Docs/CISI.ALL
CISI.ALL : 1460 items, over 108747 lines.
../Research Work/Docs/CISI.BLN
CISI.BLN : 0 items, over 86 lines.
../Research Work/Docs/CISI.QRY
CISI.QRY : 112 items, over 1525 lines.
../Research Work/Docs/CISI.REL
CISI.REL : 76 items, over 3114 lines.


In [3]:
with open('../Research Work/Docs/CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
# print n lines
n = 5
for l in lines[:n]:
    print(l)

.I 1
.T 18 Editions of the Dewey Decimal Classifications
.A Comaromi, J.P.
.W The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad.
.X 1	5	1 92	1	1 262	1	1 556	1	1 1004	1	1 1024	1	1 1024	1	1


In [4]:
doc_set = {}
doc_id = ""
doc_text = ""
for l in lines:
    if l.startswith(".I"):
        doc_id = l.split(" ")[1].strip()
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.

# Print something to see the dictionary structure, etc.
print(f"Number of documents = {len(doc_set)}" + ".\n")
print(doc_set["3"]) # note that the dictionary indexes are strings, not numbers. 

Number of documents = 1460.

Two Kinds of Power An Essay on Bibliographic Control Wilson, P. The relationships between the organization and control of writings and the organization and control of knowledge and information will inevitably enter our story, for writings contain, along with much else, a great deal of mankind's stock of knowledge and information.  Bibliographical control is a form of power, and if knowledge itself is a form of power, as the familiar slogan claims, bibliographical control is in a certain sense power over power, power to obtain the knowledge recorded in written form.  As writings are not simply, and not in any simple way, storehouses of knowledge, we cannot satisfactorily discuss bibliographical control as simply control over the knowledge and information contained in writings. 


In [5]:
with open('../Research Work/Docs/CISI.QRY') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = l.split(" ")[1].strip()
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""
        
# Print something to see the dictionary structure, etc.
print(f"Number of queries = {len(qry_set)}" + ".\n")
print(qry_set["3"]) # note that the dictionary indexes are strings, not numbers. 

Number of queries = 112.

What is information science?  Give definitions where possible.


In [6]:
rel_set = {}
with open('../Research Work/Docs/CISI.REL') as f:
    for l in f.readlines():
        qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]
        doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]
        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)
        if qry_id == "7":
            print(l.strip("\n"))
    
# Print something to see the dictionary structure, etc.
print(f"\nNumber of mappings = {len(rel_set)}" + ".\n")
print(rel_set["3"]) # note that the dictionary indexes are strings, not numbers. 

     7    310	0	0.000000
     7    320	0	0.000000
     7    332	0	0.000000
     7    375	0	0.000000
     7    376	0	0.000000
     7    645	0	0.000000
     7    724	0	0.000000
     7    725	0	0.000000

Number of mappings = 76.

['60', '85', '114', '123', '126', '131', '133', '136', '138', '140', '346', '359', '363', '372', '412', '445', '454', '461', '463', '469', '532', '537', '540', '553', '554', '555', '585', '590', '599', '640', '660', '664', '803', '901', '909', '911', '1027', '1053', '1169', '1179', '1181', '1190', '1191', '1326']


In [37]:
list1=[]
list1 = list(doc_set.values())

In [78]:
qry_set
queries = list(qry_set.values())

len(qry_set)

112

In [39]:
list1 = list1+queries
len(list1)

1572

In [40]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.stem import WordNetLemmatizer

def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def lemma(data):
    lemmatizer = WordNetLemmatizer() 
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + lemmatizer.lemmatize(w)
    return new_text

def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = lemma(data)
    data = remove_punctuation(data)
    data = lemma(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    #data = word_tokenize(data)
    return data

In [41]:
for i, sentence in enumerate(list1):
	tokenized= []
	tokenized.append(preprocess(sentence))
	list1[i] = tokenized

In [42]:
for i in range(len(list1)):
    for sentence in list1[i]:
        tokenized= []
        tokenized.append(word_tokenize(sentence))
        list1[i] = tokenized

In [43]:
output = []
for sublist in list1:
    for item in sublist:
        output.append(item)

In [44]:
from gensim.models import word2vec

In [46]:
model = word2vec.Word2Vec(output, size=100, window=8, min_count=1, workers=4) 

In [47]:
from gensim.similarities import WmdSimilarity
num_best = 1460
instance = WmdSimilarity(output, model, num_best=1460)

In [15]:
qry_set
queries = list(qry_set.values())

print(qry_set)

{'1': 'What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?', '2': 'How can actually pertinent data, as opposed to references or entire articles themselves, be retrieved automatically in response to information requests?', '3': 'What is information science?  Give definitions where possible.', '4': 'Image recognition and any other methods of automatically transforming printed text into computer-ready form.', '5': 'What special training will ordinary researchers and businessmen need for proper information management and unobstructed use of information retrieval systems? What problems are they likely to encounter?', '6': 'What possibilities are there for verbal communication between computers and humans, that is, communication via the spoken word?', '7': 'Describe presently working and planned systems for publ

In [79]:
sentence = queries[25]
arr2 = []
arr2.append(sentence)

for i, sentence in enumerate(arr2):
	tokenized= []
	tokenized.append(preprocess(sentence))
	arr2[i] = tokenized
    
for i in range(0, len(arr2)):
    for sentence in arr2[i]:
        tokenized= []
        tokenized.append(word_tokenize(sentence))
        arr2[i] = tokenized
        
output2 = []
for sublist in arr2:
    for item in sublist:
        output2.append(item)
        
output3 = []
for sublist in output2:
    for item in sublist:
        output3.append(item) 
        
output3

['cost',
 'determination',
 'cost',
 'associated',
 'system',
 'automated',
 'information']

In [80]:
sims = instance[output3]

In [81]:
sims

[(1485, 1.0),
 (1479, 0.9836021148850475),
 (1492, 0.9821654406220719),
 (1484, 0.9812688595282009),
 (1487, 0.9812450339177203),
 (1472, 0.9806484627317821),
 (1482, 0.9782668761521734),
 (1481, 0.9777984332624895),
 (174, 0.9773690727851417),
 (1099, 0.9773272218104434),
 (1474, 0.9773213640044889),
 (178, 0.9765964981268639),
 (1469, 0.976457654144443),
 (1486, 0.9764033734228961),
 (1514, 0.9763727270766122),
 (1494, 0.976220550740236),
 (1491, 0.9755329070865248),
 (134, 0.9752830743773065),
 (1281, 0.9749840052548011),
 (1490, 0.9745161732544432),
 (1480, 0.9744826513622258),
 (1109, 0.9742930690348948),
 (458, 0.9740883307960468),
 (1160, 0.9738597028643865),
 (1483, 0.9735933810275224),
 (1462, 0.9730882346924481),
 (1478, 0.9724989167067287),
 (1565, 0.9724880198061918),
 (1091, 0.9724310444533798),
 (1488, 0.9722700736772942),
 (1404, 0.9720573080653283),
 (1088, 0.9720040251210342),
 (1501, 0.9716664844353897),
 (1447, 0.9716601868749613),
 (1500, 0.9716308074047697),
 (594,

In [84]:
res=[]
for i in range(0,260):
    if(sims[i][0] < 1460):
        res.append(sims[i][0] +1)

In [85]:
len(res)

201

In [86]:
rel1=[]
rel1= rel_set["26"]
for i in range(0, len(rel1)): 
    rel1[i] = int(rel1[i])
    
print(rel1)

[17, 27, 36, 67, 74, 83, 126, 158, 164, 192, 214, 218, 307, 338, 364, 365, 372, 381, 446, 458, 465, 468, 474, 490, 491, 495, 496, 497, 500, 507, 523, 534, 584, 591, 615, 620, 639, 675, 690, 723, 732, 779, 813, 822, 848, 849, 860, 865, 979, 1100, 1264, 1366, 1368, 1374, 1376, 1377]


In [87]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

In [88]:
x = len(intersection(res, rel1))
y = len(rel1)

In [89]:
x

12

In [90]:
y

56

In [91]:
x/y * 100

21.428571428571427